In [1]:
import librosa
import torch
from torch import nn
import torch.nn.functional as F
from torchaudio import load
import torchaudio
from scipy.io.wavfile import read, write
import torchaudio
from librosa.util import normalize
from librosa.filters import mel as librosa_mel_fn
import numpy as np
import librosa
import argparse
import librosa.display
from tqdm import tqdm
import os
import soundfile as sf
import matplotlib.pyplot as plt
import parser


/data/v-leyzhang/conda/fairseq/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from examples.textless_nlp.dgslm.dgslm_utils import HubertTokenizer
encoder = HubertTokenizer(
    hubert_path = "/data/v-leyzhang/dGSLM/hubert_fisher.pt",
    hubert_layer = 12,
    km_path = "/data/v-leyzhang/dGSLM/hubert_fisher_km_500.bin"
)

INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /home/v-leyzhang
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/ntuanh/datasets/Fisher/fisher-vad-10s-separated/manifest', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/ntuanh/experiments/dialogue-LM/hubert/fisher-vad-10s-separated/kmeans/hubert_iter2/km500', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': tran

In [3]:

MAX_WAV_VALUE = 32768.0

def load_wav(full_path):
    sampling_rate, data = read(full_path)
    return data, sampling_rate

def dynamic_range_compression(x, C=1, clip_val=1e-5):
    return np.log(np.clip(x, a_min=clip_val, a_max=None) * C)

def dynamic_range_decompression(x, C=1):
    return np.exp(x) / C

def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):
    return torch.log(torch.clamp(x, min=clip_val) * C)

def dynamic_range_decompression_torch(x, C=1):
    return torch.exp(x) / C

def spectral_normalize_torch(magnitudes):
    output = dynamic_range_compression_torch(magnitudes)
    return output

def spectral_de_normalize_torch(magnitudes):
    output = dynamic_range_decompression_torch(magnitudes)
    return output

mel_basis = {}
hann_window = {}

def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis:
        mel = librosa_mel_fn(sr=sampling_rate, n_fft=n_fft, n_mels=num_mels, fmin=fmin, fmax=fmax)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)

    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                      center=center, pad_mode='reflect', normalized=False, onesided=True,return_complex=False)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)
    spec = spectral_normalize_torch(spec)

    return spec


In [4]:
path = "/data/v-leyzhang/dGSLM/val_only_laughter/11298-A/fe_03_11298-A_045.wav"
wav, _ = load(path)
print("wav", wav.shape)
mel = mel_spectrogram(wav, n_fft=480, num_mels=80, sampling_rate=_,
                        hop_size=160, win_size=480, fmin=0, fmax=4000)

print("mel",mel.shape)
plt.imshow(mel.squeeze().cpu().numpy(), origin='lower')

RuntimeError: Failed to open the input "/data/v-leyzhang/dGSLM/val_only_laughter/11298-A/fe_03_11298-A_045.wav" (No such file or directory).

In [6]:
file_list = ["777_126732_000076_000008_8k_16k.wav","LJ001-0001_8k_16k.wav"]
for filename in file_list:
    path_16k = "/home/v-leyzhang/demo/conversation_TTS/20231126-conversation-demo/"
    codes = encoder.wav2code(os.path.join(path_16k,filename),1)
    codes = codes.split(" ")
    hubert_codes = np.array(codes)
    np.save(os.path.join("/home/v-leyzhang/demo/conversation_TTS/20231126-conversation-demo", filename.replace(".wav","_hubert_code")), hubert_codes)


In [ ]:
checkpoint_file
config_file = os.path.join(os.path.split(a.checkpoint_file)[0], 'config.json')
with open(config_file) as f:
    data = f.read()

global h
json_config = json.loads(data)
h = AttrDict(json_config)

torch.manual_seed(h.seed)
global device
if torch.cuda.is_available():
    torch.cuda.manual_seed(h.seed)
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

inference(a)